# Model tuning
- Interactive tuning
- Parameters selection
- Clear solution pipeline

In [1]:
# Modules import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn import linear_model, metrics, preprocessing

from time import time
from tqdm import tqdm

# Settings
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
sns.set(style="white")

In [2]:
data_validation = pd.read_csv('./results/dataset_for_validation.csv').drop('index', axis=1)
data_full = pd.read_csv('./results/dataset_for_test.csv').drop('category_0', axis=1)
val_answers = pd.read_csv('./results/validation_answers.csv', header=None)

In [3]:
submision_sample = pd.read_csv('./readonly/sample_submission.csv')

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import mean_squared_error as mse

Few words should be said about unpacking our data. To separate our test and train data we use "date_block_num" value, but then we should make a feature out of it by making a number of month out of it.

Validation, methods comparison.

In [5]:
X_train = data_validation[data_validation['date_block_num'] < 33].drop('target', axis=1)
X_test = data_validation[data_validation['date_block_num'] == 33].drop('target', axis=1)
y_train = data_validation[data_validation['date_block_num'] < 33]['target']
y_test = val_answers[1]

X_train['date_block_num'] %= 12
X_test['date_block_num'] %= 12

**Basic lgbm**. It was vey helpful to use it to find out if dataset was built correctly without leakages and with needed features.

In [6]:
time_start = time()
lgbm = LGBMRegressor(n_estimators=100, n_jobs=2)
lgbm.fit(X_train, y_train)

print("mse is", mse(y_test, lgbm.predict(X_test.fillna(0))))
print("Time spent:", time() - time_start)

mse is 1.5237335160776573
Time spent: 0.8367640972137451


**Bagging model**.

In [7]:
class BaggingModel():
    def __init__(self, arguments: list, name, verbose=False):
        self.bag_of_models = []
        self.name = name
        self.verbose=verbose
        
        for pair in arguments:
            if 'lgbm' == pair[0]:
                self.bag_of_models.append(LGBMRegressor(**pair[1]))

            if 'knn' == pair[0]:
                self.bag_of_models.append(KNeighborsRegressor(**pair[1]))
                
        
    def predict(self, X_test):
        answer = np.zeros(X_test.shape[0])
        
        if self.verbose:
            print(self.name, "is predicting...")
            for model in tqdm(self.bag_of_models):
                answer +=  model.predict(X_test)
            print(self.name, "finished predicting")
            
        else:
            for model in self.bag_of_models:
                answer +=  model.predict(X_test)
            
        answer /= len(self.bag_of_models)
        
        return answer
        
        
    def fit(self, X_train, y_train):    
        if self.verbose:
            print(self.name, "is fitting...")
            for model in tqdm(self.bag_of_models):
                model.fit(X_train, y_train)
            print(self.name, "finished fitting")
                
        else:
            for model in self.bag_of_models:
                model.fit(X_train, y_train) 

In [8]:
time_start = time()
model = BaggingModel([('lgbm', {'n_estimators': 64, 'n_jobs': 8, 'random_state': 42, 'num_leaves': 64}), 
                       ('lgbm', {'n_estimators': 128, 'n_jobs': 8, 'random_state': 42, 'num_leaves': 32}),
                       ('lgbm', {'n_estimators': 128, 'n_jobs': 8, 'random_state': 42, 'num_leaves': 16})
                     ], 'LGBM_Bagging', True)
model.fit(X_train, y_train)

print("mse is", mse(y_test, model.predict(X_test.fillna(0))))
print("Time spent:", time() - time_start)

LGBM_Bagging is fitting...


100%|████████████████████████████████████████████| 3/3 [00:01<00:00,  1.62it/s]


LGBM_Bagging finished fitting
LGBM_Bagging is predicting...


100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 23.50it/s]


LGBM_Bagging finished predicting
mse is 1.5130420752105236
Time spent: 1.9966630935668945


**Stacking**.

In [9]:
class StackingModel():
    def __init__(self, name):
        self.name = name
        self.levels = []
        
    def append(self, models: list):
        assert models != [], "Input is empty"
        self.levels.append(models)
    
    def predict(self, X_test):
        assert self.levels != [], "Model is empty"
        assert self.levels[-1], "Model has wrong output"
        
        print(self.name, "is predicting...")
        
        level_output = X_test
        for level in tqdm(self.levels):
            level_output = np.concatenate([
                model.predict(level_output).reshape(-1, 1) for model in level
            ], axis=1)
        
        print(self.name, "finished predicting")
        
        return level_output.reshape(-1, 1)
        
    def fit(self, X_train, y_train):
       
        assert self.levels != [], "Model is empty"
        assert self.levels[-1], "Model has wrong output"
        
        print(self.name, "is fitting...")
        
        level_output = X_train
        for level in tqdm(self.levels):
            for model in level:
                model.fit(level_output, y_train)
            
            level_output = np.concatenate([
                model.predict(level_output).reshape(-1, 1) for model in level
            ], axis=1)

        print(self.name, "finished fitting")

In [10]:
time_start = time()
model = StackingModel("Stacking Machine")
model.append([LGBMRegressor(n_jobs=8), LGBMRegressor(n_jobs=8), LGBMRegressor(n_jobs=8)])
model.append([LGBMRegressor(n_jobs=8), LGBMRegressor(n_jobs=8)])
model.append([KNeighborsRegressor()])
model.fit(X_train, y_train)

print("mse is", mse(y_test, model.predict(X_test.fillna(0))))
print("Time spent:", time() - time_start)

Stacking Machine is fitting...


100%|████████████████████████████████████████████| 3/3 [00:03<00:00,  1.15s/it]


Stacking Machine finished fitting
Stacking Machine is predicting...


100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 13.31it/s]


Stacking Machine finished predicting
mse is 1.5313531353135315
Time spent: 3.684155225753784


**Solvator-2000**

In [17]:
time_start = time()

# Model building
bag1 = BaggingModel([('lgbm', {'n_estimators': 64, 'n_jobs': 8, 'random_state': 42, 'num_leaves': 64}), 
                       ('lgbm', {'n_estimators': 128, 'n_jobs': 8, 'random_state': 42, 'num_leaves': 32}),
                       ('lgbm', {'n_estimators': 128, 'n_jobs': 8, 'random_state': 42, 'num_leaves': 16})
                     ], 'LGBM_Bagging', False)
bag2 = BaggingModel([('knn', {'leaf_size': 64, 'n_neighbors': 50, 'n_jobs': 8}), 
                       ('knn', {'leaf_size': 32, 'n_neighbors': 100, 'n_jobs': 8}), 
                       ('knn', {'leaf_size': 64, 'n_neighbors': 10, 'n_jobs': 8})
                     ], 'kNN_Bagging', False)
model = StackingModel("Solvator-2000")
model.append([bag1, bag2])
model.append([SGDRegressor(), RandomForestRegressor(n_estimators=128, n_jobs= 8, random_state=42)])
model.append([LinearRegression()])

# Model fit
model.fit(X_train, y_train)

print("mse is", mse(y_test, model.predict(X_test.fillna(0)).clip(0, 20)))
print("Time spent:", time() - time_start)

Solvator-2000 is fitting...


100%|████████████████████████████████████████████| 3/3 [00:17<00:00,  5.75s/it]


Solvator-2000 finished fitting
Solvator-2000 is predicting...


100%|████████████████████████████████████████████| 3/3 [00:01<00:00,  2.94it/s]


Solvator-2000 finished predicting
mse is 0.692785511745209
Time spent: 18.311983108520508


For submission we chose Solvator-2000 because of the cool name.

In [19]:
X_train_subm = data_full[data_full['date_block_num'] < 34].drop('target', axis=1)
X_test_subm = data_full[data_full['date_block_num'] == 34].drop('target', axis=1)
y_train_subm = data_full[data_full['date_block_num'] < 34]['target']

X_train_subm['date_block_num'] %= 12
X_test_subm['date_block_num'] %= 12

In [20]:
time_start = time()

# Model building
bag1 = BaggingModel([('lgbm', {'n_estimators': 64, 'n_jobs': 8, 'random_state': 42, 'num_leaves': 64}), 
                       ('lgbm', {'n_estimators': 128, 'n_jobs': 8, 'random_state': 42, 'num_leaves': 32}),
                       ('lgbm', {'n_estimators': 128, 'n_jobs': 8, 'random_state': 42, 'num_leaves': 16})
                     ], 'LGBM_Bagging', False)
bag2 = BaggingModel([('knn', {'leaf_size': 64, 'n_neighbors': 50, 'n_jobs': 8}), 
                       ('knn', {'leaf_size': 32, 'n_neighbors': 100, 'n_jobs': 8}), 
                       ('knn', {'leaf_size': 64, 'n_neighbors': 10, 'n_jobs': 8})
                     ], 'kNN_Bagging', False)
model = StackingModel("Solvator-2000")
model.append([bag1, bag2])
model.append([SGDRegressor(), RandomForestRegressor(n_estimators=128, n_jobs= 8, random_state=42)])
model.append([LinearRegression()])

# Model fit
model.fit(X_train_subm, y_train_subm)

print("Time spent:", time() - time_start)

Solvator-2000 is fitting...


100%|████████████████████████████████████████████| 3/3 [00:25<00:00,  8.56s/it]


Solvator-2000 finished fitting
Time spent: 25.68553352355957


In [41]:
data_full[data_full['target'] == 0].shape

(227674, 189)

In [42]:
data_full[data_full['target'] != 0].shape

(1130, 189)

In [21]:
submision_sample['item_cnt_month'] = model.predict(X_test_subm).clip(0, 20)
submision_sample.to_csv('./results/submission.csv', index=False)

Solvator-2000 is predicting...


100%|████████████████████████████████████████████| 3/3 [02:08<00:00, 42.96s/it]


Solvator-2000 finished predicting


In [26]:
print("mse is", mse(y_test, model.predict(X_test.fillna(0))))

Solvator-2000 is predicting...


  0%|                                                    | 0/3 [00:00<?, ?it/s]


ValueError: Number of features of the model must match the input. Model n_features_ is 188 and input n_features is 155 